## Import Libraries

In [11]:
import retro

In [6]:
#Search for ROMS
!python -m retro.import .

Importing StreetFighterIISpecialChampionEdition-Genesis
Imported 1 games


In [13]:
#Loads the game env
env=retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

AttributeError: module 'retro' has no attribute 'make'

In [8]:
#Close the env
env.close()

NameError: name 'env' is not defined

In [ ]:
#See the posible imputs that it could have
env.action_space.sample()

array([0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1], dtype=int8)

In [15]:
#Put game in first state
obs = env.reset()
#Flag
done = False
for game in range(1):
    while not done:
        if done:
            obs=env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        print(reward)

SyntaxError: invalid syntax (com.py, line 200)